In [ ]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

from backtesting import Strategy
from backtesting import Backtest

In [ ]:
# Read in SPY data from last 18 years
dfSPY=yf.download("SPY",start='2005-01-01', end='2023-07-31')
dfSPY.head()

# Calculate Simple Moving Average and Relative Strength Index and add to dataframe
dfSPY['SMA']=ta.sma(dfSPY.Close, length=200)
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=4)

# Add Bollinger Band values (upper/lower) to the dataframe
my_bbands = ta.bbands(dfSPY.Close, length=20, std=2.5)
dfSPY=dfSPY.join(my_bbands)

# Remove rows with no data on 200 day moving average
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)

In [ ]:
# Function to add SMA trend signal
def addsmasignal(df, backcandles):
    smasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.SMA[i]:
                dnt=0
            if df.Low[i]<=df.SMA[i]:
                upt=0
        if upt==1 and dnt==1:
            smasignal[row]=3
        elif upt==1:
            smasignal[row]=2
        elif dnt==1:
            smasignal[row]=1
    df['SMASignal'] = smasignal

addsmasignal(dfSPY, 5)

In [ ]:
# Add orders to dataframe
def addorderslimit(df):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if (df.SMASignal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]) or (df.SMASignal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]):
            ordersignal[i]=df.Close[i]
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY)

dfSPY[dfSPY['ordersignal'] > 0]

# Visualization

In [ ]:
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [ ]:
# Backtesting
class MyStrat(Strategy):
    initsize = 0.99
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            if self.data.index[-1]-self.ordertime[0]>5:
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.SMASignal==2:
            # Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            # Add replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.SMASignal==1:
            # Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            # Add replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)
stat = bt.run()
stat

In [ ]:
bt.plot(show_legend=False)